In [1]:
import os
import sys
from dotmap import DotMap

sys.path.append('../../')

from utils.io import recursive_scan2df, load_yaml
from utils.sampling import ConsecutiveSequences

In [2]:
server = 'local'
server = DotMap(load_yaml('../../config/servers.yml')[server])

database_name = 'cholec80'
database_df = recursive_scan2df(os.path.join(server.database.location, database_name), postfix='.mp4')
database_df

,file,folder
0,video36.mp4,videos
1,video37.mp4,videos
2,video52.mp4,videos
3,video38.mp4,videos
4,video53.mp4,videos
...,...,...
79,video35.mp4,videos
80,video01_short.mp4,sample_videos
81,video02_short.mp4,sample_videos
82,video03_short.mp4,sample_videos


In [3]:
# individual database post-processing
# database_df = database_df[database_df.folder != 'videos']
database_df = database_df[database_df.folder != 'sample_videos']
# database_df = database_df.iloc[:1]
database_df

,file,folder
0,video36.mp4,videos
1,video37.mp4,videos
2,video52.mp4,videos
3,video38.mp4,videos
4,video53.mp4,videos
...,...,...
75,video33.mp4,videos
76,video34.mp4,videos
77,video50.mp4,videos
78,video51.mp4,videos


In [4]:
from datasets import VideoDataset
from utils.io import load_pickle, save_pickle

video_paths = [os.path.join(server.database.location, database_name, x.folder, x.file) for _, x in database_df.iterrows()]
frame_rate = 25
duration = 60
clip_length_in_frames = int(frame_rate*duration)
num_workers = 8

# load metadata, if existing
metadata_path = 'cholec80_metadata.pkl'
metadata = None
print('\nLooking for {}'.format(metadata_path))
if os.path.exists(metadata_path):
    metadata = load_pickle(metadata_path)
    print('Found.')
else:
    print('Could not find {}. Generating metadata.'.format(metadata_path))

video_ds = VideoDataset(
            video_paths=video_paths,
            clip_length_in_frames=clip_length_in_frames,
            frames_between_clips=clip_length_in_frames,
            frame_rate=frame_rate,
            precomputed_metadata=metadata,
            num_workers=num_workers,
            permute=False,
            convert_dtype=False
)

# store metadata
if not os.path.exists(metadata_path):
    metadata = video_ds.metadata
    save_pickle(metadata_path, metadata)

import cv2
import numpy as np
from utils.processing import endoscopy

# buffer = []
# buffer_size = 10

for batch in video_ds:
    video, augmented_video, frame_rate, video_fps, video_idx, idx = batch

    for frame in video:
        # frame = tensor_to_image(frame)
        frame = frame.numpy()
        # frame = frame[:-5,...]

        # pre-process frame
        mask = endoscopy.bilateralSegmentation(frame, th=0.1)
        # buffer.append(mask)
        # if len(buffer) > buffer_size:
        #     buffer.pop(0)
        # mask = endoscopy.binaryAvg(buffer, th=50)
        center, radius = endoscopy.boundary_detection.ransacBoundaryCircle(mask, fit='numeric', n_iter=1)

        if radius is not None:
            illumination = endoscopy.illuminationLevel(mask, center, radius)
            print('\rIllumination level: {}, threshold: {}'.format(illumination, illumination > 0.998), end='')

            top_left, shape = endoscopy.maxRectangleInCircle(frame.shape, center, radius)
            center, radius = center.astype(np.int), np.int(radius)
            top_left, shape = top_left.astype(np.int), tuple(map(int, shape))
            cv2.circle(frame, (center[1], center[0]), radius, (255, 255, 0), 2)
            cv2.rectangle(frame, (top_left[1], top_left[0]), (top_left[1] + shape[1], top_left[0] + shape[0]), (255, 255, 0), 2)

        cv2.imshow('img', frame)
        cv2.imshow('mask', mask)
        # cv2.waitKey(int(1/frame_rate*1.e3))
        cv2.waitKey(1)

    print(video.shape)
cv2.destroyAllWindows()


Looking for cholec80_metadata.pkl
Found.
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
Illumination level: 0.9772400896464106/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/ipykernel_launcher.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/martin/miniconda3/envs/hil/lib/python3.7/site-package